In [ ]:
import pandas as pd
from datetime import datetime
import mydashtools as dt
import plotly.express as px
pd.options.plotting.backend = "plotly"

zelty = pd.read_csv("../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)
zelty['date'] = pd.to_datetime(zelty['date'])
if dt.is_notebook():
    print(zelty.head())

In [ ]:
chiffredaffaire = zelty.groupby(['date', 'category'])[['price']].sum().unstack('category')

In [ ]:
chiffredaffaire_par_an = chiffredaffaire.resample('YS').sum().fillna(0)
chiffredaffaire_par_an['year'] = chiffredaffaire_par_an.index.year
chiffredaffaire_par_an.reset_index(inplace=True)
chiffredaffaire_par_an.set_index('year', inplace=True)
# = chiffredaffaire.columns.droplevel(0)
chiffredaffaire_par_an.columns = chiffredaffaire_par_an.columns.droplevel(0)
del chiffredaffaire_par_an['']

fig = px.bar(chiffredaffaire_par_an)
dt.fig_save_or_show(fig, 'chiffresdaffaire_01_ca_par_categories_et_an')

In [ ]:
chiffredaffaire_12_mois = chiffredaffaire.resample('MS').sum()
chiffredaffaire_12_mois = chiffredaffaire_12_mois[chiffredaffaire_12_mois.index > str(datetime.now().replace(year=datetime.now().year - 1))]
chiffredaffaire_12_mois.columns = chiffredaffaire_12_mois.columns.droplevel(0)

fig = px.bar(chiffredaffaire_12_mois)
dt.fig_save_or_show(fig, 'chiffresdaffaire_02_ca_par_categories_12_mois')

In [ ]:
chiffredaffaire_total_by_year = chiffredaffaire.stack('category').reset_index().groupby('date')[['price']].sum().resample('Y').sum()
chiffredaffaire_total_by_year.index = chiffredaffaire_total_by_year.index.year

chiffredaffaire_a_date = chiffredaffaire.stack('category').reset_index().groupby('date')[['price']].sum()
chiffredaffaire_a_date['a_date'] = chiffredaffaire_a_date.index.month*100+chiffredaffaire_a_date.index.day
chiffredaffaire_a_date = chiffredaffaire_a_date[chiffredaffaire_a_date['a_date'] < datetime.now().month*100+datetime.now().day]
del chiffredaffaire_a_date['a_date']
chiffredaffaire_a_date = chiffredaffaire_a_date.resample('Y').sum()
chiffredaffaire_a_date.index = chiffredaffaire_a_date.index.year
chiffredaffaire_a_date
libelle_a_date = "à date du %02d/%02d" % (datetime.now().day, datetime.now().month)
chiffredaffaire_total_by_year[libelle_a_date] = chiffredaffaire_a_date['price']
chiffredaffaire_total_by_year.fillna(0, inplace=True)
chiffredaffaire_total_by_year['toute l\'année'] = chiffredaffaire_total_by_year['price'] - chiffredaffaire_total_by_year[libelle_a_date]
del chiffredaffaire_total_by_year['price']

fig = px.bar(chiffredaffaire_total_by_year)
dt.fig_save_or_show(fig, 'chiffresdaffaire_03_ca_total_par_an')

In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
chiffredaffaire_year = chiffredaffaire[chiffredaffaire.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
chiffredaffaire_year.columns = chiffredaffaire_year.columns.droplevel(0)
chiffredaffaire_year = chiffredaffaire_year.groupby('date').sum()
chiffredaffaire_year['total'] = pd.DataFrame(chiffredaffaire_year.stack('category'), columns=['total']).groupby('date').sum()
chiffredaffaire_top_year = chiffredaffaire_year.sort_values(by='total', ascending=False)

if dt.is_notebook():
    print(chiffredaffaire_top_year[['total']].head(20))


In [ ]:
fig = px.bar(chiffredaffaire_year['total'])
dt.fig_save_or_show(fig, 'chiffresdaffaire_04_ca_quotidien_12_mois')

In [ ]:
del chiffredaffaire_top_year['total']
chiffredaffaire_top_year.index = chiffredaffaire_top_year.index.strftime('%d/%m/%Y')

fig = px.bar(chiffredaffaire_top_year.head(50))
dt.fig_save_or_show(fig, 'chiffresdaffaire_05_ca_quotidien_top50_12_mois')

In [ ]:
marges = {2017:0.46,
          2018:0.4795,
          2019:0.5012,
          2020:0.3679,
          2021:0.5078,
          2022:0.3363,
          2023:0.45}


In [ ]:
charges = {2017:50353,
           2018:50353,
           2019:50786,
           2020:45973-33171,
           2021:30298-40000,
           2022:45886,
           2023:45886
          }


In [ ]:
benefices = chiffredaffaire.stack('category')[['price']].groupby('date').sum().resample('MS').sum()
benefices['year'] = benefices.index.year
benefices['marge'] = benefices['year'].map(marges)
benefices['charges_fixes'] = benefices['year'].map(charges)/12
benefices = benefices.groupby(['date','year']).sum()
benefices['taux_ht'] = 0.8333
benefices['total'] = benefices['price'] * benefices['marge'] * benefices['taux_ht'] - benefices['charges_fixes']
del benefices['price']
del benefices['marge']
del benefices['charges_fixes']
del benefices['taux_ht']
benefices = benefices.unstack('year').fillna(0)
benefices.columns = benefices.columns.droplevel(0)

fig = px.bar(benefices)
dt.fig_save_or_show(fig, 'chiffresdaffaire_06_benefice')

In [ ]:
if dt.is_notebook():
    print(benefices.sum())